In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [2]:
train_x = pd.read_csv("data/train.csv")
train_x_indexes = train_x["PassengerId"].count()
train_y = train_x["Survived"]
test = pd.read_csv("data/test.csv")
all_data = pd.concat([train_x, test])
all_data.reset_index(inplace=True, drop=True) 

## Inspecting the dataset

In [3]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [4]:
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
print("index column count: "+ str(all_data["PassengerId"].count()))
print(all_data.isna().sum())       

index column count: 1309
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


In [6]:
all_data_2 = all_data.drop(["Survived","PassengerId", "Ticket", "Cabin"], axis=1) #handling null values in Cabin column doesent worth to dealing with, so much missing data
                                                                                      #I'm going to use name column later
passenger_id_col = all_data["PassengerId"]

In [7]:
all_data_2.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [8]:
all_data_2.nunique()

Pclass         3
Name        1307
Sex            2
Age           98
SibSp          7
Parch          8
Fare         281
Embarked       3
dtype: int64

## Dealing with null values

In [9]:
age_mean = round(all_data_2["Age"].mean())

In [10]:
all_data_2["Age"] = all_data_2["Age"].fillna(age_mean)
all_data_2["Embarked"] = all_data_2["Embarked"].ffill()
all_data_2["Fare"] = all_data_2["Fare"].ffill()

In [11]:
print(all_data_2.isna().sum())  

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


## Altering columns

In [12]:
numeric_cols = all_data_2[["Age", "SibSp", "Parch", "Fare"]]
numeric_cols.head()

,Age,SibSp,Parch,Fare
0,22.0,1,0,7.2500
1,38.0,1,0,71.2833
2,26.0,0,0,7.9250
3,35.0,1,0,53.1000
4,35.0,0,0,8.0500


In [13]:
one_hot_encode_cols = all_data_2[["Pclass", "Sex", "Embarked"]]
one_hot_encode_cols.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [27]:
one_hot_encode_cols[["Pclass"]] = one_hot_encode_cols[["Pclass"]].astype(str)
one_hot_encode_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    1309 non-null   object
 1   Sex       1309 non-null   object
 2   Embarked  1309 non-null   object
dtypes: object(3)
memory usage: 30.8+ KB


In [15]:
name_tags=[]
for index in range(all_data_2["Name"].count()):
    name_tags.append(all_data_2["Name"][index].split(", ")[1].split(".")[0])
all_data_2.drop(["Name"], axis=1, inplace=True)
series = pd.Series(name_tags, name='Name_tags')
one_hot_encode_cols_2 = pd.concat([one_hot_encode_cols, series], axis =1)
one_hot_encode_cols_2

,Pclass,Sex,Embarked,Name_tags
0,3,male,S,Mr
1,1,female,C,Mrs
2,3,female,S,Miss
3,1,female,S,Mrs
4,3,male,S,Mr
...,...,...,...,...
1304,3,male,S,Mr
1305,1,female,C,Dona
1306,3,male,S,Mr
1307,3,male,S,Mr


In [16]:
one_hot_encoded_df = pd.get_dummies(one_hot_encode_cols_2)
one_hot_encoded_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Name_tags_Capt,Name_tags_Col,...,Name_tags_Master,Name_tags_Miss,Name_tags_Mlle,Name_tags_Mme,Name_tags_Mr,Name_tags_Mrs,Name_tags_Ms,Name_tags_Rev,Name_tags_Sir,Name_tags_the Countess
0,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
1,True,False,False,True,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,True,True,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,True,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
1305,True,False,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1306,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
1307,False,False,True,False,True,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False


In [17]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(numeric_cols)
numeric_col_scaled = pd.DataFrame(scaled_data, columns=numeric_cols.columns)
numeric_col_scaled

,Age,SibSp,Parch,Fare
0,0.273456,0.125,0.000000,0.014151
1,0.473882,0.125,0.000000,0.139136
2,0.323563,0.000,0.000000,0.015469
3,0.436302,0.125,0.000000,0.103644
4,0.436302,0.000,0.000000,0.015713
...,...,...,...,...
1304,0.373669,0.000,0.000000,0.015713
1305,0.486409,0.000,0.000000,0.212559
1306,0.480145,0.000,0.000000,0.014151
1307,0.373669,0.000,0.000000,0.015713


In [18]:
full_data_x = pd.concat([one_hot_encoded_df,numeric_col_scaled], axis =1 )
pd.options.display.max_columns = None
full_data_x.head()

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Name_tags_Capt,Name_tags_Col,Name_tags_Don,Name_tags_Dona,Name_tags_Dr,Name_tags_Jonkheer,Name_tags_Lady,Name_tags_Major,Name_tags_Master,Name_tags_Miss,Name_tags_Mlle,Name_tags_Mme,Name_tags_Mr,Name_tags_Mrs,Name_tags_Ms,Name_tags_Rev,Name_tags_Sir,Name_tags_the Countess,Age,SibSp,Parch,Fare
0,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.273456,0.125,0.0,0.014151
1,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.473882,0.125,0.0,0.139136
2,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0.323563,0.000,0.0,0.015469
3,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.436302,0.125,0.0,0.103644
4,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.436302,0.000,0.0,0.015713


In [19]:
final_data = full_data_x.iloc[:train_x_indexes, : ]
test_data = full_data_x.iloc[train_x_indexes: , : ]
test_data.reset_index(inplace=True, drop=True)

## Creating a simple neural network

In [20]:
import tensorflow as tf

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(final_data, train_y, epochs=25)

Epoch 1/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5383 - loss: 0.6807
Epoch 2/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7920 - loss: 0.5165
Epoch 3/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8075 - loss: 0.4578
Epoch 4/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8218 - loss: 0.4371
Epoch 5/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8345 - loss: 0.4099
Epoch 6/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8328 - loss: 0.4050
Epoch 7/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8439 - loss: 0.3905
Epoch 8/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8444 - loss: 0.3910
Epoch 9/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8500 - loss: 0.3794
Epoch 10/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8291 - loss: 0.3958
Epoch 11/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8399 - loss: 0.3752
Epoch 12/25
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8521 - lo

In [23]:
preds = model.predict(test_data)
final_preds = (preds > 0.5).astype(int)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [24]:
reshape = final_preds.reshape(-1)
reshaped = pd.Series(reshape, name = "Survived")
final_subm = pd.concat([test["PassengerId"], reshaped], axis = 1)
final_subm

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [25]:
final_subm.to_csv("preds.csv", index=False)

In [26]:
final_subm

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
